In [1]:
from argparse import Namespace
import os
import sys

os.chdir("/playpen-nas-ssd/luchao/projects/SAM/")

import pprint
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms
from torchvision.utils import make_grid, save_image
import torchvision.transforms.functional as F
import time
import torch.nn as nn
import copy
import shutil
import collections

# from criteria.aging_loss import AgingLoss
from criteria.aging_loss_fpage import AgingLoss



aging_loss = AgingLoss(opts=None).cuda()
# predict_age = aging_loss.extract_ages(result_tensor.unsqueeze(0)).item()

def iterate_over_img_dir(img_dir):
    IMG_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']
    for root, dirs, files in os.walk(img_dir):
        for file in files:
            if any(file.endswith(extension) for extension in IMG_EXTENSIONS):
                yield os.path.join(root, file)

/playpen-nas-ssd/gongbang/miniconda3/envs/arc2face/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/playpen-nas-ssd/luchao/projects/SAM/criteria/aging_loss_fpage.py:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization

In [2]:
def calculate_age_error(img_dir):
    results_age_pred = collections.defaultdict(list)
    for img_path in iterate_over_img_dir(img_dir):
        img_name = os.path.basename(img_path)
        # if img_name.count('_') == 1:
        #     # print(f'Skip input image: {img_name}')
        #     continue
        if any([keyword in img_name for keyword in ['inversion']]):
            # print(f'Skip inversion image: {img_name}')
            continue
        img = Image.open(img_path).convert('RGB') # (3, 1024, 1024)
        img_tensor = F.to_tensor(img).unsqueeze(0).cuda() # (1, 3, 1024, 1024)
        predicted_age = aging_loss.extract_ages(img_tensor).item()
        target_age = int(img_name.split('.')[0].split('_')[-1])
        assert 0 <= target_age <= 100, f'Invalid age: {target_age}'
        results_age_pred[target_age].append(predicted_age)
    age_errors = []
    for target_age, pred_ages in results_age_pred.items():
        # ----------------------------------- todo ----------------------------------- #
        # if target_age > 70:
        #     # * used for benchmarking against other baselines as some baselines do not support > 70
        #     continue
        age_errors.append(abs(np.mean(pred_ages) - target_age))
    return np.mean(age_errors)
    # return age_errors

In [3]:
# ours = []
# mytm = []
fading = []
# flowedit = []
# rfsolver = []
rfinversion = []
fireflow = []

In [6]:
celebs = ['al', 'charles', 'chow', 'diego', 'elaine', 'elizabeth', 'jackie', 'jennifer', 'nicole', 'oprah', 'robert', 'thatcher']

for celeb in celebs:
    exp_dir_parent = f'/playpen-nas-ssd/gongbang/comparisons/fireflow/{celeb}'
    # exp_dir_parent = f'/playpen-nas-ssd/gongbang/personalized/{celeb}'

    for exp in sorted(os.listdir(exp_dir_parent)):
        exp_dir = os.path.join(exp_dir_parent, exp)
        if not os.path.exists(exp_dir):
            print(f'{exp_dir} does not exist')
            continue
        # ! change here
        # if not f'pretrained_{celeb}_70' in exp_dir:
        #     continue
        # if not f'pretrained_{celeb}_40' in exp_dir:
        #     continue
        # if not '2_55' in exp_dir:
        #     continue

        # single_img_dir = os.path.join(exp_dir_parent, exp, 'single_img')
        age_error = calculate_age_error(exp_dir)
        print(f'Experiment: {exp_dir}')
        print(f'Age error: {age_error}')
        print('-'*10)
        fireflow.append(age_error)

Experiment: /playpen-nas-ssd/gongbang/comparisons/fireflow/al/alcoholism
Age error: 15.214771840479468
----------
Experiment: /playpen-nas-ssd/gongbang/comparisons/fireflow/al/goodskin
Age error: 18.761850703846328
----------
Experiment: /playpen-nas-ssd/gongbang/comparisons/fireflow/al/hairloss
Age error: 18.969927081814056
----------
Experiment: /playpen-nas-ssd/gongbang/comparisons/fireflow/al/poorskin
Age error: 15.989009820021593
----------
Experiment: /playpen-nas-ssd/gongbang/comparisons/fireflow/al/weight
Age error: 16.97173056664405
----------
Experiment: /playpen-nas-ssd/gongbang/comparisons/fireflow/charles/alcoholism
Age error: 16.638951956820325
----------
Experiment: /playpen-nas-ssd/gongbang/comparisons/fireflow/charles/goodskin
Age error: 17.927996577048788
----------
Experiment: /playpen-nas-ssd/gongbang/comparisons/fireflow/charles/hairloss
Age error: 19.07364632050737
----------
Experiment: /playpen-nas-ssd/gongbang/comparisons/fireflow/charles/poorskin
Age error: 16

In [7]:
np.mean(fireflow)

16.45411384602266

In [12]:
ours

14.671840257165785

In [ ]:
ours = [a / 14 for a in ours]
mytm_new = [a / 14 for a in mytm_new]
fading_pers = [a / 14 for a in fading_pers]
fading_db = [a / 14 for a in fading_db]

In [ ]:
import matplotlib.pyplot as plt

def plot_dicts_per_key(dicts, x_values):
    """
    Plots lists from multiple dictionaries, creating one plot per key.

    :param dicts: List of dictionaries with the same keys.
    :param x_values: List of x-axis values corresponding to the lists.
    """
    if not dicts:
        print("No data to plot.")
        return

    # Extract keys from the first dictionary (assumes all dictionaries share the same keys)
    plt.figure(figsize=(10, 6))

    labels = ['ours', 'MyTM', 'FADING Pers ft', 'FADING DB']
    
    for i in range(len(dicts)):
        values = dicts[i]

        corresponding_x = x_values

        # Plot the values for the current dictionary
        plt.plot(corresponding_x, values, marker='o', label=f"{labels[i]}")

    # Add labels, legend, and grid
    plt.xlabel("Target Age (10-90)")
    plt.ylabel("Age MAE")
    plt.title(f"Comparison")
    plt.legend(title="Source")
    plt.grid(True)
    plt.show()

# Example usage
x_values = [10, 20, 30, 40, 50, 60, 70, 80, 90]

# Call the function with the dictionaries and x-axis values
plot_dicts_per_key([ours, mytm_new, fading_pers, fading_db], x_values)

In [ ]:
ours = np.mean(ours)
mytm_new = np.mean(mytm_new)
fading_pers = np.mean(fading_pers)
fading_db = np.mean(fading_db)